In [69]:
%pip install google-generativeai pdf2image pytesseract pillow pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [70]:
import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, Optional, Tuple
from dotenv import load_dotenv
from jsonschema import validate, ValidationError
from supabase import create_client, Client

print("✓ All libraries imported")

✓ All libraries imported


In [71]:
PDF_DIR = r"C:\Users\samue\Documents\Work\Code\valuation_data_miner\data"
OUTPUT_DIR = "./extracted_data"
ERROR_LOG_DIR = "./errors"

# Load environment variables
dotenv_path = r"C:\Users\samue\Documents\Work\Code\valuation_data_miner\.env"
load_dotenv(dotenv_path)

# API Keys
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Model settings
GEMINI_MODEL = "models/gemini-2.5-flash" 
TESSERACT_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Create directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(ERROR_LOG_DIR, exist_ok=True)

print("✓ Configuration loaded")
print(f"  PDF Directory: {PDF_DIR}")
print(f"  Gemini Model: {GEMINI_MODEL}")

if not GEMINI_API_KEY:
    print("\n⚠ WARNING: GEMINI_API_KEY not found!")
    print("  Get key from: https://makersuite.google.com/app/apikey")

✓ Configuration loaded
  PDF Directory: C:\Users\samue\Documents\Work\Code\valuation_data_miner\data
  Gemini Model: models/gemini-2.5-flash


In [72]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content("Hello, respond with a JSON: {\"test\": 123}", 
                                  generation_config=genai.types.GenerationConfig(
                                      temperature=0.1,
                                      max_output_tokens=200
                                  ))

print(response.text)


```json
{"test": 123}
```


In [73]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)

models = genai.list_models()
for m in models:
    print(m.name)


models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-l

In [74]:
VALUATION_SCHEMA = {
    "type": "object",
    "required": ["property_id", "valuation_report", "property_details", "valuations"],
    "properties": {
        "property_id": {"type": "string"},
        "valuation_report": {
            "type": "object",
            "properties": {
                "report_reference": {"type": "string"},
                "valuer": {"type": "string"},
                "valuers": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "qualification": {"type": "string"}
                        }
                    }
                },
                "inspection_date": {"type": "string"},
                "report_date": {"type": "string"},
                "client": {"type": "string"},
                "client_address": {"type": "string"},
                "purpose": {"type": "string"}
            }
        },
        "property_details": {
            "type": "object",
            "properties": {
                "apartment_number": {"type": "string"},
                "block": {"type": "string"},
                "floor": {"type": "string"},
                "title_details": {"type": "object"},
                "location": {"type": "object"},
                "tenure": {"type": "object"},
                "registered_proprietors": {"type": "array"},
                "ownership_type": {"type": "string"},
                "encumbrances": {"type": "string"}
            }
        },
        "property_description": {"type": "object"},
        "apartment_details": {"type": "object"},
        "occupancy": {"type": "string"},
        "condition": {"type": "string"},
        "market_assessment": {"type": "object"},
        "valuations": {
            "type": "object",
            "properties": {
                "current_market_value": {
                    "type": "object",
                    "properties": {
                        "amount": {"type": "number"},
                        "currency": {"type": "string"},
                        "amount_words": {"type": "string"}
                    }
                }
            }
        },
        "valuation_methodology": {"type": "array"},
        "lease_details": {"type": "object"},
        "compliance": {"type": "object"}
    }
}

print("✓ JSON schema defined")

✓ JSON schema defined


In [75]:
def validate_extracted_data(data: Dict) -> Tuple[bool, Optional[str]]:
    """Validate extracted data against schema"""
    try:
        validate(instance=data, schema=VALUATION_SCHEMA)
        return True, None
    except ValidationError as e:
        return False, str(e)

print("✓ Validation function defined")


✓ Validation function defined


In [76]:
def init_supabase() -> Optional[Client]:
    """Initialize Supabase client"""
    try:
        if not SUPABASE_URL or not SUPABASE_KEY:
            print("⚠ Supabase credentials not configured")
            return None
        
        client = create_client(SUPABASE_URL, SUPABASE_KEY)
        print("✓ Supabase client initialized")
        return client
    except Exception as e:
        print(f"✗ Supabase init failed: {e}")
        return None

supabase_client = init_supabase()


✓ Supabase client initialized


In [77]:
def upload_to_supabase(client: Client, data: Dict, filename: str) -> bool:
    """Upload to Supabase"""
    if client is None:
        return False
    
    try:
        upload_data = data.copy()
        upload_data['source_filename'] = filename
        upload_data['processed_at'] = datetime.now().isoformat()
        
        client.table("property_valuations").insert(upload_data).execute()
        return True
    except Exception as e:
        raise Exception(f"Supabase upload failed: {str(e)}")

print("✓ Supabase functions defined")

✓ Supabase functions defined


In [78]:
def process_scanned_pdf_with_ocr(pdf_path: str) -> str:
    """Convert scanned PDF to text using Tesseract OCR"""
    try:
        from pdf2image import convert_from_path
        import pytesseract
        
        # Set Tesseract path
        if os.path.exists(TESSERACT_PATH):
            pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH
        else:
            raise Exception(f"Tesseract not found at {TESSERACT_PATH}. Install from: https://github.com/UB-Mannheim/tesseract/wiki")
        
        print(f"    → Converting PDF to images...")
        
        # Convert PDF to images
        images = convert_from_path(pdf_path, dpi=300, fmt='jpeg', thread_count=2)
        
        print(f"    → Running OCR on {len(images)} pages...")
        
        # OCR each page
        all_text = []
        for i, image in enumerate(images):
            print(f"      → Page {i+1}/{len(images)}...", end=' ')
            
            text = pytesseract.image_to_string(
                image,
                lang='eng',
                config='--psm 1'
            )
            
            text = text.strip()
            
            if text:
                all_text.append(f"--- Page {i+1} ---\n{text}")
                print(f"✓ {len(text)} chars")
            else:
                print("(empty)")
        
        full_text = "\n\n".join(all_text)
        
        print(f"    → OCR complete: {len(full_text)} characters")
        
        # Save OCR output
        ocr_file = os.path.join(ERROR_LOG_DIR, f"{Path(pdf_path).stem}_ocr_text.txt")
        with open(ocr_file, 'w', encoding='utf-8') as f:
            f.write(full_text)
        
        return full_text
        
    except Exception as e:
        raise Exception(f"OCR failed: {str(e)}")

print("✓ OCR function defined")

✓ OCR function defined


In [79]:
def process_pdf_with_fallback(pdf_path: str) -> str:
    """Try multiple extraction methods"""
    
    # Try pdfplumber first
    try:
        import pdfplumber
        print(f"    → Trying pdfplumber...")
        
        all_text = []
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    all_text.append(f"--- Page {i+1} ---\n{text}")
        
        full_text = "\n\n".join(all_text)
        
        if len(full_text) > 500:
            print(f"    ✓ pdfplumber: {len(full_text)} chars")
            return full_text
        else:
            print(f"    → Only {len(full_text)} chars, trying OCR...")
    except Exception as e:
        print(f"    → pdfplumber failed, trying OCR...")
    
    # Use OCR
    return process_scanned_pdf_with_ocr(pdf_path)

print("✓ PDF extraction with fallback defined")

✓ PDF extraction with fallback defined


In [107]:
import os
import time
import json
from pathlib import Path
from typing import List
from datetime import datetime
import google.generativeai as genai

# -----------------------------
# CONFIGURATION
# -----------------------------
PDF_DIR = r"C:\Users\samue\Documents\Work\Code\valuation_data_miner\data"
OUTPUT_DIR = "./extracted_data"
ERROR_LOG_DIR = "./errors"

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = "models/gemini-2.5-flash"  # safer, available model

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(ERROR_LOG_DIR, exist_ok=True)

if not GEMINI_API_KEY:
    raise Exception("GEMINI_API_KEY not found in environment")

genai.configure(api_key=GEMINI_API_KEY)

# -----------------------------
# HELPERS
# -----------------------------
def chunk_text(text: str, max_chars: int = 12000) -> List[str]:
    """Split long text into smaller chunks."""
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunks.append(text[start:end])
        start = end
    return chunks

def clean_gemini_json(response_text: str) -> str:
    """Extract JSON from Gemini response."""
    if '```json' in response_text:
        response_text = response_text.split('```json')[1].split('```')[0]
    elif '```' in response_text:
        response_text = response_text.split('```')[1].split('```')[0]
    response_text = response_text.strip()
    if not response_text.startswith('{'):
        start = response_text.find('{')
        end = response_text.rfind('}')
        if start != -1 and end != -1:
            response_text = response_text[start:end+1]
    return response_text

def deep_merge(d1, d2):
    """Recursively merge two dictionaries."""
    for k, v in d2.items():
        if (
            k in d1 and isinstance(d1[k], dict) and isinstance(v, dict)
        ):
            d1[k] = deep_merge(d1[k], v)
        else:
            d1[k] = v
    return d1

# -----------------------------
# EXTRACTION
# -----------------------------
def extract_with_gemini(text_content: str, filename: str) -> dict:
    """Extract structured data from a document with chunking and retries."""
    chunks = chunk_text(text_content, max_chars=12000)
    combined_result = {}

    for i, chunk in enumerate(chunks):
        prompt = f"""Extract property valuation data from this Kenyan document (OCR text). Return ONLY valid JSON.

RULES:
- Extract EXACT values from document (no placeholders)
- If field not found, use "" or {{}}
- Amounts as numbers without commas
- Dates as YYYY-MM-DD

Schema:
{{
  "property_id": "string",
  "valuation_report": {{"report_reference": "", "valuer": "", "valuers": [], "inspection_date": "",
                        "report_date": "", "client": "", "client_address": "", "purpose": ""}},
  "property_details": {{"apartment_number": "", "block": "", "floor": "",
                        "title_details": {{}}, "location": {{}}, "tenure": {{}} ,
                        "registered_proprietors": [], "ownership_type": "", "encumbrances": ""}},
  "property_description": {{}} ,
  "apartment_details": {{}} ,
  "occupancy": "",
  "condition": "",
  "market_assessment": {{}} ,
  "valuations": {{"current_market_value": {{"amount": 0, "currency": "KES", "amount_words": ""}}}} ,
  "valuation_methodology": [] ,
  "lease_details": {{}} ,
  "compliance": {{}}
}}

Document:
{chunk}

Return ONLY JSON:"""

        response_text = ""
        success = False
        for attempt in range(3):
            try:
                model = genai.GenerativeModel(GEMINI_MODEL)
                start_time = time.time()
                response = model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=0.1,
                        max_output_tokens=8192,
                    )
                )
                elapsed = time.time() - start_time

                if hasattr(response, "text") and response.text:
                    response_text = clean_gemini_json(response.text)
                    extracted_data = json.loads(response_text)
                    combined_result = deep_merge(combined_result, extracted_data)
                    print(f"Chunk {i+1}/{len(chunks)} succeeded in {elapsed:.1f}s, attempt {attempt+1}")
                    success = True
                    break
            except json.JSONDecodeError as e:
                print(f"Chunk {i} JSON decode failed, attempt {attempt+1}: {e}")
            except Exception as e:
                print(f"Chunk {i} Gemini error, attempt {attempt+1}: {e}")

        if not success:
            print(f"⚠ Chunk {i} failed after 3 attempts")

        # Save response for debugging
        debug_file = os.path.join(ERROR_LOG_DIR, f"{Path(filename).stem}_gemini_response_chunk{i+1}.txt")
        with open(debug_file, 'w', encoding='utf-8') as f:
            f.write(response_text or "")

    # Ensure property_id exists
    if not combined_result.get("property_id"):
        combined_result["property_id"] = Path(filename).stem

    return combined_result


In [81]:
def process_single_pdf_complete(pdf_path: str, filename: str) -> dict:
    """Complete pipeline: OCR → Gemini → Validate → Save"""
    result = {
        'filename': filename,
        'success': False,
        'stage': None,
        'error': None,
        'data': None,
        'timing': {}
    }
    
    start_time = time.time()
    
    try:
        # Stage 1: OCR
        result['stage'] = 'ocr'
        print(f"\n  📄 Stage 1: OCR")
        stage_start = time.time()
        
        text_content = process_pdf_with_fallback(pdf_path)
        result['timing']['ocr'] = time.time() - stage_start
        
        if len(text_content) < 200:
            raise Exception(f"Insufficient text: {len(text_content)} chars")
        
        print(f"  ✓ {len(text_content)} chars in {result['timing']['ocr']:.1f}s")
        
        # Stage 2: Gemini
        result['stage'] = 'gemini'
        print(f"\n  🤖 Stage 2: Gemini")
        stage_start = time.time()
        
        extracted_data = extract_with_gemini(text_content, filename)
        result['timing']['gemini'] = time.time() - stage_start
        result['data'] = extracted_data
        
        print(f"  ✓ Extracted in {result['timing']['gemini']:.1f}s")
        
        # Stage 3: Validation
        result['stage'] = 'validation'
        print(f"\n  ✅ Stage 3: Validation")
        
        is_valid, validation_error = validate_extracted_data(extracted_data)
        if not is_valid:
            print(f"  ⚠ Warning: {validation_error[:100]}")
        else:
            print(f"  ✓ Valid")
        
        # Stage 4: Save
        result['stage'] = 'saving'
        print(f"\n  💾 Stage 4: Save")
        output_file = os.path.join(OUTPUT_DIR, f"{Path(filename).stem}.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, indent=2, ensure_ascii=False)
        print(f"  ✓ Saved to {Path(output_file).name}")
        
        # Stage 5: Supabase
        if supabase_client and is_valid:
            result['stage'] = 'supabase'
            try:
                upload_to_supabase(supabase_client, extracted_data, filename)
                print(f"  ✓ Uploaded to Supabase")
            except Exception as e:
                print(f"  ⚠ Supabase: {str(e)[:50]}")
        
        result['success'] = True
        result['timing']['total'] = time.time() - start_time
        
    except Exception as e:
        result['error'] = str(e)
        result['timing']['total'] = time.time() - start_time
        
        error_file = os.path.join(ERROR_LOG_DIR, f"{Path(filename).stem}_error.txt")
        with open(error_file, 'w', encoding='utf-8') as f:
            f.write(f"Stage: {result['stage']}\nError: {result['error']}\n")
        
        print(f"\n  ✗ Failed: {result['error'][:100]}")
    
    return result

print("✓ Complete pipeline defined")


✓ Complete pipeline defined


In [106]:
def process_all_pdfs_complete():
    """Process all PDFs in the PDF_DIR."""
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))
    if not pdf_files:
        print(f"No PDFs found in {PDF_DIR}")
        return []

    print(f"\n{'='*70}")
    print(f"🚀 PROCESSING {len(pdf_files)} PDFs")
    print(f"{'='*70}")

    results = []
    total_start = time.time()

    for idx, pdf_file in enumerate(pdf_files, 1):
        print(f"\n{'='*70}")
        print(f"[{idx}/{len(pdf_files)}] {pdf_file.name}")
        print(f"{'='*70}")

        # OCR text extraction (you already have your OCR function)
        text_content = process_scanned_pdf_with_ocr(str(pdf_file)) 

        # Gemini extraction
        data = extract_with_gemini(text_content, pdf_file.name)

        # Save JSON
        output_file = os.path.join(OUTPUT_DIR, f"{Path(pdf_file).stem}.json")
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)

        result = {
            "filename": pdf_file.name,
            "success": bool(data),
            "data": data,
            "timing": {"total": 0},  # optionally add timing
        }
        results.append(result)

        print(f"✓ Saved to {output_file}")

    total_time = time.time() - total_start
    print(f"\nBatch completed in {total_time/60:.1f} min")

    # Save batch summary
    summary_file = os.path.join(OUTPUT_DIR, "summary.json")
    with open(summary_file, "w", encoding="utf-8") as f:
        json.dump({
            "timestamp": datetime.now().isoformat(),
            "model": GEMINI_MODEL,
            "total_pdfs": len(pdf_files),
            "results": results,
        }, f, indent=2)

    return results

In [83]:
def test_single_pdf():
    """Test with first PDF"""
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))
    if not pdf_files:
        print("No PDFs found")
        return
    
    test_file = pdf_files[0]
    print(f"\n🧪 TESTING: {test_file.name}\n")
    
    result = process_single_pdf_complete(str(test_file), test_file.name)
    
    if result['success']:
        print(f"\n✅ TEST PASSED!")
        print(f"\nPreview:")
        print(json.dumps(result['data'], indent=2)[:1000])
    else:
        print(f"\n❌ TEST FAILED")
    
    return result


print("\n" + "="*70)
print("="*70)
test_single_pdf()
# print("or:  results = process_all_pdfs_complete()")



🧪 TESTING: doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.pdf


  📄 Stage 1: OCR
    → Trying pdfplumber...
    → Only 0 chars, trying OCR...
    → Converting PDF to images...
    → Running OCR on 12 pages...
      → Page 1/12... ✓ 1266 chars
      → Page 2/12... ✓ 2401 chars
      → Page 3/12... ✓ 165 chars
      → Page 4/12... ✓ 182 chars
      → Page 5/12... ✓ 1357 chars
      → Page 6/12... ✓ 1800 chars
      → Page 7/12... ✓ 1664 chars
      → Page 8/12... ✓ 539 chars
      → Page 9/12... ✓ 491 chars
      → Page 10/12... ✓ 581 chars
      → Page 11/12... ✓ 1039 chars
      → Page 12/12... ✓ 607 chars
    → OCR complete: 12297 characters
  ✓ 12297 chars in 52.2s

  🤖 Stage 2: Gemini
Chunk 1/1 responded in 29.7s, attempt 1
  ✓ Extracted in 29.7s

  ✅ Stage 3: Validation
  ⚠ Warning: 'Leasehold' is not of type 'object'

Failed validating 'type' in schema['properties']['property_deta

  💾 Stage 4: Save
  ✓ Saved to doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MA

{'filename': 'doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.pdf',
 'success': True,
 'stage': 'saving',
 'error': None,
 'data': {'property_id': 'L.R. NUMBER: 7815/83 (I.R NUMBER: 237281)',
  'valuation_report': {'report_reference': 'SOO/DOO/5296/1/25',
   'valuer': 'Simon Oruka Orwa',
   'valuers': ['Ken Wathome', 'Nahashon Kuria', 'Simon Oruka Orwa'],
   'inspection_date': '14th April 2021',
   'report_date': '18 April 2021',
   'client': 'Mr. Kenneth Wathome',
   'client_address': 'Nairobi',
   'purpose': 'advisory purposes'},
  'property_details': {'apartment_number': '',
   'block': '',
   'floor': '',
   'title_details': {'title_number': 'IR.237281',
    'land_reference_number': '7815/83',
    'term': '999 years',
    'annual_ground_rent': '30,258.00',
    'deed_plan_number': 'F/R No. 507/87',
    'survey_plan_number': 'F/R No. 507/87, 453369'},
   'location': {'county': 'Machakos County',
    'area': 'Lukenya Area',
    'coordinates': '1°28\'12.2"S 37°02\'21.7"

In [ ]:
results = process_all_pdfs_complete()
results


🚀 PROCESSING 47 PDFs

[1/47] doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.pdf
    → Converting PDF to images...
    → Running OCR on 12 pages...
      → Page 1/12... ✓ 1266 chars
      → Page 2/12... ✓ 2401 chars
      → Page 3/12... ✓ 165 chars
      → Page 4/12... ✓ 182 chars
      → Page 5/12... ✓ 1357 chars
      → Page 6/12... ✓ 1800 chars
      → Page 7/12... ✓ 1664 chars
      → Page 8/12... ✓ 539 chars
      → Page 9/12... ✓ 491 chars
      → Page 10/12... ✓ 581 chars
      → Page 11/12... ✓ 1039 chars
      → Page 12/12... ✓ 607 chars
    → OCR complete: 12297 characters
Chunk 1/2 succeeded in 21.6s, attempt 1
Chunk 2/2 succeeded in 5.5s, attempt 1
✓ Saved to ./extracted_data\doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.json

[2/47] doc39404020250117161224 (1) TITLE NO KARAI-GIKAMBURA-6448 ONDIRI AREA KIAMBU COUNTY.pdf
    → Converting PDF to images...
    → Running OCR on 21 pages...
      → Page 1/21... ✓ 1370 chars
      → Page 2/21...